# **Mount**

Mount the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Image**

The class Image is used to contain all the necessary informations(id,features,label) associated to a picture

In [ ]:
class Image:

  def __init__(self,id,features, label = -2):
    self.id = id
    self.features = features
    self.label = label
  
  def get_id(self):
    return self.id
  
  def get_label(self):
    return self.label
  
  def get_features(self):
    return self.features

# **Hash Function**

HashFunction is the structure implementing a g function.
Its parameters are:

1. k = number of sub-hash functions
2. X[i] = hyperplane correspondig to the i-sub-hash function
3. d = len(X[i])
4. w = size of the segment = 4
5. b[i] = offset of the i-segment
6. m = mean of the Gaussian for extracting the planes
7. s = std of the Gaussian for extracting the planes

The index function returns the key identificator associated to the value passed as argument; the key is formed by the return of the various hash function applied to the value.

In [ ]:
import math

class HashFunction:

  def __init__(self, k, d, m, s, w = 4 ):
    self.k = k
    self.m = m
    self.d = d
    self.s = s
    self.w = w

    self.b = []
    self.X = []
    for i in range(k):
      #b has a value between 0 and w
      self.b.append( np.random.uniform(0, w))
    self.X = np.array([np.random.normal(loc=m[i],scale=s[i],size=k) for i in range(d)])
    self.X = np.transpose(self.X)
    
  def hash(self, value, i):
    return int( (np.dot(self.X[i], value) + self.b[i])/self.w )

  def index( self, value, max_h_function = math.inf):
    if( len( value ) != self.d  ):
      return False
    dim = min(self.k, max_h_function)
    return "".join([str(self.hash(value,i)) for i in range(dim)])

# **Hash Table**

We will create each hash table as a dictionary. Each line of the table will have all the ids of the image whose features share the projection on the same hyperplane.
We will have as many hash table as the number of g function we want to be implemented.
Each HashTable has its own g function used to obtain the hash key

In [ ]:
class HashTable:

  def __init__(self,hash_size,input_dim,mean,std):
    self.table = dict()
    self.g_function = HashFunction(hash_size,input_dim,mean,std)
    self.keysCountByG = np.zeros(7)
    self.keysOfG = [[] for _ in range(7)]
  
  def add_value(self, number_h_functions, id, features, label = -1):
    image = Image(id,features, label)
    for num_h in number_h_functions: 
      hash_key = self.g_function.index(features, num_h)
      if hash_key not in self.table:
        self.table[ hash_key ] = []
        self.keysCountByG[num_h-2] += 1
        self.keysOfG[num_h-2].append(hash_key)
      self.table[ hash_key ].append(image)

  def get_values_of_key(self, features, number_h_function):
    hash_key = self.g_function.index(features,number_h_function)
    if hash_key not in self.table:
      return False
    else:
      return self.table[ hash_key ]

  def return_keys(self):
    return self.keysCountByG
  
  def return_bucketsByH(self):
    #dim = 1
    dim = len(self.keysOfG)
    bucketsDim = [[] for _ in range(dim)]
    for i in range(dim):
      keys = self.keysOfG[i]
      for key in keys:
        bucketsDim[i].append(len(self.table[key]))
    return bucketsDim



# **LSHash**

LSHash is the structure implementing the lsh index.
Its parameters are:

1.   hash_size: number of the hyperplane implementing each g function
2.   num_hash_tables: number of g function, and consequently hash tables
3.   input_dim: dimension of the features(and of the vector representing each hyperplane)



In [ ]:
from numpy import dot
from numpy.linalg import norm
import math

class LSHash:
  def __init__(self,num_g_functions,input_dim,hash_size,mean,std, distance = "euclidean"):

    self.hash_size = hash_size
    self.num_g_functions = num_g_functions
    self.input_dim = input_dim #128
    self.init_hash_tables(hash_size,input_dim,mean,std)
    self.distance = distance
    self.number_h_functions = [2,3,4,5,6,7,8]
    #self.number_h_functions = [5]
    #self.number_h_functions = [4,8,16,32]
    #self.number_h_functions = [2,3,4,6,8] 
    #self.number_h_functions = [5,6,7] 
    #self.number_h_functions = [7,8]
  

  def print_info(self):
    print( "LSH Information:")
    print( "Hash size: " + str(self.hash_size ) )
    print( "Num g functions: " + str(self.num_g_functions ) )
    print( "Input Dim: " + str(self.input_dim ) )
    print( "Distance type: " + str(self.distance ) )
  
  def return_keys_gFunction(self,number_g):
    keys = []
    for i in range(number_g):
      keys.append(self.hash_tables[i].return_keys())
    return keys
  
  def return_bucketsDimByG(self,number_g):
    bDim = []
    for i in range(number_g):
      
      bDim.append(self.hash_tables[i].return_bucketsByH())
    return bDim
    
  #hash_tables will contain the HashTable mapped by each g function
  def init_hash_tables(self,hash_size,input_dim,mean,std):
    self.hash_tables = [HashTable(hash_size,input_dim,mean,std) for _ in range(self.num_g_functions)]
  
  def insert_features(self,id, features, label = -3):

    for i in range(self.num_g_functions):
      self.hash_tables[i].add_value(self.number_h_functions, id, features, label)
      
  def eval_distance(self, actual_features, target_feature):
    if(self.distance == "euclidean"):
      return np.linalg.norm(actual_features - target_feature)
    elif( self.distance == "cosine"):
#      return (norm(actual_features)*norm(target_feature))/dot(actual_features,target_feature)
      cos = dot(actual_features,target_feature)/(norm(actual_features)*norm(target_feature))
      return -( cos - 1 )
    else:
      return -1
  
  def insertion_point(self,potential_results,candidate,desired_results):
    index = 0
    if (len(potential_results)==0): #list potential result vuota
      return 0
    for result in potential_results:
      if ( candidate[0] == result[0] and candidate[1] == result[1]): #risultato già presente
        return -1
      elif (candidate[2] < result[2]): #ho trovato dove inserire il nuovo risultato
        return index
      else:
        index += 1
    if (index>=desired_results):#distanza maggiore tutti elementi nei results,e lista piena
      return -1
    else:
      return index



  #We find the ids of the images that share the for each g function, all the time, the same hashed key of the features of the image
  def search(self,features,desired_g_functions,desired_h_functions,desired_results = math.inf ):

    potential_results = []

    # for each hash table we retrieve the id of the images that share the same hyperplane of the features that we are projecting

    dim = min(self.num_g_functions, desired_g_functions)
    element_inserted = 0

    for i in range(dim):
      bucket = self.hash_tables[i].get_values_of_key(features,desired_h_functions)
      if bucket == False:
        continue
      for b in bucket:
        element = [b.get_id(),b.get_label(), self.eval_distance(features, b.get_features())]
        insertion = self.insertion_point(potential_results,element,desired_results)
        if (insertion==-1):
          continue
        else:
          potential_results.insert(insertion,element)
          if (element_inserted >= desired_results):
            potential_results.pop()
          else:
            element_inserted += 1

    return potential_results
